# La Anónima Price Tracker - Analysis Notebook

This notebook provides interactive analysis of scraped price data from La Anónima supermarket.

## Contents
1. [Setup and Data Loading](#setup)
2. [Price Time Series](#prices)
3. [Basket Index Calculation](#index)
4. [CPI Comparison](#cpi)
5. [Product-level Analysis](#products)

## 1. Setup and Data Loading <a name="setup"></a>

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from decimal import Decimal

from src.config_loader import load_config
from src.models import get_engine, get_session_factory, Product, Price, ScrapeRun, BasketIndex
from src.analysis import BasketAnalyzer

# Configuration
config = load_config('../config.yaml')

# Setup plotting
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
%matplotlib inline

In [ ]:
# Connect to database
engine = get_engine(config)
Session = get_session_factory(engine)
session = Session()

# Load price data
query = session.query(
    Price.canonical_id,
    Price.basket_id,
    Price.product_name,
    Price.current_price,
    Price.original_price,
    Price.in_stock,
    Price.is_promotion,
    Price.scraped_at,
).join(ScrapeRun, Price.run_id == ScrapeRun.id)

df = pd.read_sql(query.statement, session.bind)
df['scraped_at'] = pd.to_datetime(df['scraped_at'])
df['year_month'] = df['scraped_at'].dt.to_period('M').astype(str)

print(f"Loaded {len(df)} price records")
print(f"Date range: {df['scraped_at'].min()} to {df['scraped_at'].max()}")
print(f"Unique products: {df['canonical_id'].nunique()}")
df.head()

## 2. Price Time Series <a name="prices"></a>

In [ ]:
# Plot price trends for selected products
selected_products = df.groupby('canonical_id')['current_price'].count().nlargest(5).index

fig, ax = plt.subplots(figsize=(14, 6))

for product_id in selected_products:
    product_df = df[df['canonical_id'] == product_id].sort_values('scraped_at')
    name = product_df['product_name'].iloc[0][:30]  # Truncate long names
    ax.plot(product_df['scraped_at'], product_df['current_price'], marker='o', label=name)

ax.set_xlabel('Date')
ax.set_ylabel('Price (ARS)')
ax.set_title('Price Trends - Top 5 Products')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Price distribution by category
latest_prices = df.sort_values('scraped_at').groupby('canonical_id').last()

fig, ax = plt.subplots(figsize=(12, 6))
latest_prices['current_price'].hist(bins=50, ax=ax, edgecolor='black')
ax.set_xlabel('Price (ARS)')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Latest Prices')
ax.axvline(latest_prices['current_price'].median(), color='red', linestyle='--', label='Median')
ax.legend()
plt.show()

print(f"Median price: ${latest_prices['current_price'].median():.2f}")
print(f"Mean price: ${latest_prices['current_price'].mean():.2f}")

## 3. Basket Index Calculation <a name="index"></a>

In [ ]:
# Compute basket index
analyzer = BasketAnalyzer(config, session)

# For CBA basket
cba_index = analyzer.compute_basket_index('cba', save_to_db=False)

if not cba_index.empty:
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Index value
    ax1 = axes[0]
    ax1.plot(cba_index['year_month'], cba_index['index_value'], marker='o', linewidth=2)
    ax1.set_xlabel('Period')
    ax1.set_ylabel('Index Value (Base = 100)')
    ax1.set_title('CBA Basket Index Over Time')
    ax1.grid(True, alpha=0.3)
    ax1.tick_params(axis='x', rotation=45)
    
    # Month-over-month change
    ax2 = axes[1]
    mom_data = cba_index.dropna(subset=['mom_change'])
    colors = ['green' if x < 0 else 'red' for x in mom_data['mom_change']]
    ax2.bar(mom_data['year_month'], mom_data['mom_change'], color=colors, alpha=0.7)
    ax2.set_xlabel('Period')
    ax2.set_ylabel('MoM Change (%)')
    ax2.set_title('Month-over-Month Inflation (CBA Basket)')
    ax2.grid(True, alpha=0.3, axis='y')
    ax2.tick_params(axis='x', rotation=45)
    ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    
    plt.tight_layout()
    plt.show()
    
    display(cba_index)
else:
    print("No CBA index data available")

## 4. CPI Comparison <a name="cpi"></a>

In [ ]:
# Load and compare with CPI data
# CPI file should have columns: year_month, cpi_index, cpi_mom, cpi_yoy

comparison = analyzer.compare_with_cpi('cba')

if not comparison.empty and 'cpi_index' in comparison.columns:
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Index comparison
    ax1 = axes[0]
    ax1.plot(comparison['year_month'], comparison['index_value'], 
             marker='o', linewidth=2, label='Basket Index (Online)')
    ax1.plot(comparison['year_month'], comparison['cpi_index'], 
             marker='s', linewidth=2, label='CPI (INDEC)')
    ax1.set_xlabel('Period')
    ax1.set_ylabel('Index Value')
    ax1.set_title('Basket Index vs Official CPI')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.tick_params(axis='x', rotation=45)
    
    # Difference
    ax2 = axes[1]
    if 'index_diff_pct' in comparison.columns:
        diff_data = comparison.dropna(subset=['index_diff_pct'])
        ax2.bar(diff_data['year_month'], diff_data['index_diff_pct'], 
                color='purple', alpha=0.7)
        ax2.set_xlabel('Period')
        ax2.set_ylabel('Difference (%)')
        ax2.set_title('Basket Index vs CPI Difference')
        ax2.grid(True, alpha=0.3, axis='y')
        ax2.tick_params(axis='x', rotation=45)
        ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    
    plt.tight_layout()
    plt.show()
    
    # Error metrics
    if hasattr(comparison, 'attrs'):
        print(f"MAE: {comparison.attrs.get('mae', 'N/A'):.2f}")
        print(f"RMSE: {comparison.attrs.get('rmse', 'N/A'):.2f}")
else:
    print("No CPI data available for comparison")

## 5. Product-level Analysis <a name="products"></a>

In [ ]:
# Calculate price changes for each product
price_changes = []

for product_id in df['canonical_id'].unique():
    product_df = df[df['canonical_id'] == product_id].sort_values('scraped_at')
    
    if len(product_df) >= 2:
        first_price = product_df['current_price'].iloc[0]
        last_price = product_df['current_price'].iloc[-1]
        change_pct = ((last_price - first_price) / first_price) * 100
        
        price_changes.append({
            'canonical_id': product_id,
            'product_name': product_df['product_name'].iloc[0],
            'first_price': first_price,
            'last_price': last_price,
            'change_pct': change_pct,
            'observations': len(product_df)
        })

changes_df = pd.DataFrame(price_changes)

# Top price increases
print("TOP 10 PRICE INCREASES:")
print(changes_df.nlargest(10, 'change_pct')[['product_name', 'change_pct', 'observations']].to_string())

print("\nTOP 10 PRICE DECREASES:")
print(changes_df.nsmallest(10, 'change_pct')[['product_name', 'change_pct', 'observations']].to_string())

In [ ]:
# Volatility analysis
volatility = []

for product_id in df['canonical_id'].unique():
    product_df = df[df['canonical_id'] == product_id].sort_values('scraped_at')
    
    if len(product_df) >= 3:
        prices = product_df['current_price']
        cv = prices.std() / prices.mean() * 100  # Coefficient of variation
        
        volatility.append({
            'canonical_id': product_id,
            'product_name': product_df['product_name'].iloc[0],
            'cv': cv,
            'mean_price': prices.mean(),
            'observations': len(product_df)
        })

vol_df = pd.DataFrame(volatility)

# Most volatile products
print("MOST VOLATILE PRODUCTS (by coefficient of variation):")
print(vol_df.nlargest(10, 'cv')[['product_name', 'cv', 'observations']].to_string())

In [ ]:
# Close session
session.close()
print("Analysis complete!")